In [1]:
# Utils
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import time
#import _pickle as pickle
import pickle
import re, sys, unidecode
#import unidecode


# Representation
import nltk
from nltk.corpus import stopwords
import scikitplot.plotters as skplt

import wordbatch
from wordbatch.extractors import WordBag, WordHash
from wordbatch.models import FTRL

from tensorflow.contrib.learn import DNNClassifier
from tensorflow.contrib.learn import DNNEstimator
#from tecnosmartlib import DataObject

/root/anaconda3/lib/python3.6/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so | _pywrap_tensorflow_internal
/root/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/lite/toco/python/_tensorflow_wrap_toco.so | _tensorflow_wrap_toco


In [2]:
#plt.style.use('fivethirtyeight')

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load Data

In [3]:
dfNoticias = pd.read_pickle("dfNoticias")

# pre procesing

Cleaning data, select clases

In [4]:
print('Selecting useful information...', end='')
_map = {}
_map["Cuerpo"] = []
_map["Seccion"] = []
_map["Seccion2"] = []
_map["Seccion3"] = []

count = 0
for index, row in dfNoticias.iterrows():
    if (row["Seccion_1"] != None and row["Cuerpo"] != None):
        _map["Cuerpo"].append(row["Cuerpo"]) 
        _map["Seccion"].append(row["Seccion_1"])
        if ( row["Seccion_2"] != None ):
            _map["Seccion2"].append(row["Seccion_2"])
        else:
            _map["Seccion2"].append(row["Seccion_1"])
        
        if ( row["Seccion_3"] != None ):
            _map["Seccion3"].append(row["Seccion_3"])
        else:
            _map["Seccion3"].append(row["Seccion_1"])
        
        
        if row["Seccion_1"] == row["Seccion_2"] and row["Seccion_2"] == row["Seccion_3"]:
            count += 1
        if row["Seccion_1"] == row["Seccion_2"] and row["Seccion_3"] == None:
            count += 1
        if row["Seccion_1"] == row["Seccion_3"] and row["Seccion_2"] == None:  
            count += 1 
        if row["Seccion_2"] == None and row["Seccion_3"] == None:
            count += 1
print('done.')



Selecting useful information...done.


In [5]:
df = pd.DataFrame(_map)
numberOfDuplicates = df.shape[0]
print('Number of data before dropping duplicates: {}'.format(df.shape[0]))
df = df.drop_duplicates(inplace= False)
df.reset_index(drop=True, inplace= True)
numberOfDuplicates -= df.shape[0]
print('Number of data after dropping duplicates : {}'.format(df.shape[0]))
print('\nNumber of duplicated data : {}'.format(numberOfDuplicates))

#df.head()

Number of data before dropping duplicates: 250339
Number of data after dropping duplicates : 238918

Number of duplicated data : 11421


In [6]:
X_untransformed = df['Cuerpo'].reset_index(drop=True)
y_untransformed = df['Seccion'].reset_index(drop=True)

y2_untransformed = df['Seccion2'].reset_index(drop=True)
y3_untransformed = df['Seccion3'].reset_index(drop=True)


assert X_untransformed.shape[0] == y_untransformed.shape[0], 'X and y dimenssion must be the same.'
print('Number of data: {}'.format(X_untransformed.shape[0]))

Number of data: 238918


In [7]:
classes = df.groupby('Seccion')['Cuerpo'].nunique()

print(classes)
nClasses = classes.shape[0]

#print('\nNumber of classes: {}'.format(nClasses))

Seccion
Corporativo        82
Cultura          3128
Deportes        76458
Economía         6750
Entretención    29438
Mundo           31664
País            75180
Sociedad         9079
Tecnología       7130
Name: Cuerpo, dtype: int64


# Feature Extraccion

### Data representation

TFID calculation

In [8]:

start = time.time()
print("start")


spanish_stopwords = stopwords.words('spanish')
def normalize_text(text):
    return u" ".join([x for x in [y for y in text.lower().strip().split(" ")] 
                      if len(x) > 1 and x not in spanish_stopwords])

X_untransformed = X_untransformed
n_docs = X_untransformed.shape[0]
n_cpu = 20

batch_size = int(n_docs/n_cpu)

wb = wordbatch.WordBatch(normalize_text, 
                         extractor=(WordBag, {"hash_ngrams": 1, "hash_ngrams_weights": [1.0, 1.0],
                                              "hash_size": 2**28, "norm": "l2", "tf": 1.0,
                                              "idf": 1.0}), procs=n_cpu, n_words=1000, minibatch_size=batch_size)
wb.dictionary_freeze = True
word_comment = wb.fit_transform(list(X_untransformed),reset= False)
X_transformed = word_comment[:, np.array(np.clip(word_comment.getnnz(axis=0) - 1, 0, 1), dtype = bool)]

end = time.time()
print("TFIDF end time :" + str(end - start) )

X = X_transformed
print('Number of features: {}'.format(X.shape[1]))

start
Normalize text
Extract wordbags
TFIDF end time :103.23783087730408
Number of features: 1000


### Output

In [9]:
class DataObject():
    def __init__(self, x, y,y2,y3):
        self.X = x
        self.Y = y
        self.Y2 = y2
        self.Y3 = y3
        
        
        try:
            self.Y.reset_index(drop=True, inplace=True)
            self.Y2.reset_index(drop=True, inplace=True)
            self.Y3.reset_index(drop=True, inplace=True)
        except:
            pass

        self.X_train = self.X
        self.Y_train = self.Y
        self.X_test  = None
        self.Y_test  = None
        self.X_test_dense = None
        self.k = 0
        
#     def next_train_batch(self, batch_size):
#         indexes = np.random.choice(self.X_train.shape[0], size=batch_size, replace=False)
#         return self.X_train[indexes], self.Y_train.reindex(indexes)

    def set_train_test(self,train_fraction = None, test_fraction = None, dense=True):
        # Tests for create subsets
        if train_fraction == None and test_fraction == None:
            train_fraction = 0.8
        elif train_fraction == None:
            assert 0 <= test_fraction and test_fraction < 1, 'Test fraction must be in [0,1)'
            train_fraction = 1 - test_fraction
        else:
            assert 0 < train_fraction and train_fraction <= 1, 'Train fraction must be in (0,1]'

        train_indices = np.random.choice(self.X.shape[0], round(train_fraction*self.X.shape[0]), replace=False)
        test_indices = np.array(list(set(range(self.X.shape[0])) - set(train_indices)))

        self.X_train = self.X[train_indices]
        self.Y_train = self.Y.reindex(train_indices)
        self.Y_train = self.Y_train.reset_index()
        self.Y2_train = self.Y2.reindex(train_indices)
        self.Y2_train = self.Y2_train.reset_index()
        self.Y3_train = self.Y3.reindex(train_indices)
        self.Y3_train = self.Y3_train.reset_index()        
        
        self.X_test = self.X[test_indices]
        self.Y_test = self.Y.reindex(test_indices)
        self.Y_test = self.Y_test.reset_index()
        self.Y2_test = self.Y2.reindex(test_indices)
        self.Y2_test = self.Y2_test.reset_index()
        self.Y3_test = self.Y3.reindex(test_indices)
        self.Y3_test = self.Y3_test.reset_index()        
        if dense:
            self.X_test_dense = self.X_test.todense()
        return True

def remove_accents(a):
    #b = list(map(lambda x:unidecode.unidecode(x), a))
    return unidecode.unidecode(a)
    #return unidecode.unidecode(b)


In [10]:
Y = y_untransformed
Y2 = y2_untransformed
Y3= y3_untransformed

Y = Y.reset_index(drop=True)
Y2 = Y2.reset_index(drop=True)
Y3 = Y3.reset_index(drop=True)

Y = Y.apply(remove_accents)
Y2 = Y2.apply(remove_accents)
Y3 = Y3.apply(remove_accents)

data = DataObject(X,Y,Y2,Y3)

labels = [remove_accents(x) for x in list(data.Y.unique())]
#print(labels)

data.set_train_test(0.8)
#print(data.Y_train.reset_index(drop=True))

True

In [16]:
set(Y)

{'Corporativo',
 'Cultura',
 'Deportes',
 'Economia',
 'Entretencion',
 'Mundo',
 'Pais',
 'Sociedad',
 'Tecnologia'}

# Model

### DNN graph generation

In [11]:
tf.set_random_seed(42)

feature_columns = [tf.contrib.layers.real_valued_column('x', dimension=1000)]
#feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

classifier = DNNEstimator(                                
                           head = tf.contrib.learn.multi_label_head(n_classes=9,weight_column_name='class_weights' ) , 
                           feature_columns=feature_columns,    
                           hidden_units=[2000, 1000, 100],
                           dropout=0.1,
                           #model_dir = './model-parallel/seccion_dnn',
                           model_dir = './tmp3',
                           config = tf.contrib.learn.RunConfig(save_checkpoints_steps = 500,
                           save_checkpoints_secs = None)
                          )

#tensorboard --logdir=./tmp2
# 10.20.0.114:6006

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9f51f5a6d8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': 500, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './tmp4'}


In [32]:
tf.set_random_seed(42)

feature_columns = [tf.contrib.layers.real_valued_column('x', dimension=1000)]
#feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]


Y_ = data.Y_train
Y_ = Y_['Seccion'].reset_index(drop=True)
y = Y_.values
label_keys_ = list(set(y))

classifier = DNNEstimator(                                
                           head = tf.contrib.learn.multi_class_head(n_classes=9,weight_column_name='class_weights',label_keys = label_keys_ ) , 
                           feature_columns=feature_columns,    
                           hidden_units=[2000, 1000, 100],
                           dropout=0.8,
                           #model_dir = './model-parallel/seccion_dnn',
                           model_dir = './tmp5',
                           config = tf.contrib.learn.RunConfig(save_checkpoints_steps = 500,                           
                           save_checkpoints_secs = None)
                          )



INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9f29f70c50>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': 500, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './tmp5'}


### functions graph tensorflow

In [30]:
epochs = 100000

# Define the test inputs
def get_train_inputs():
    print("Paso por aca.........................")
    X_ = data.X_train
    X_ = X_.todense()
    x = X_

    data.Y2_train["Seccion2"] = data.Y2_train["Seccion2"].replace(to_replace="Goles", value='Deportes')
    
    Y_ = data.Y_train
    Y2_ = data.Y2_train    
    Y3_ = data.Y3_train
    Y_ = Y_['Seccion'].reset_index(drop=True)
    Y2_ = Y2_['Seccion2'].reset_index(drop=True)
    Y3_ = Y3_['Seccion3'].reset_index(drop=True)
    
    y = Y_.values
    y2 = Y2_.values
    y3 = Y3_.values
    #print(type(y))
    
    # computing lengths of each class
    sizes = {label: Y_[Y_ == label].shape[0] for label in labels}

    # creating weights for each sample
    scale_factor = 10e3
    weights = np.asarray([1.0/sizes[label] for label in y])
    weights = scale_factor*weights
    weights = weights[:,np.newaxis]
    #print(weights)
    
    #print('weights shape: {}'.format(weights.shape))
    
    
#     labels2 = list( set(y) )

#     def to_multi_index(x1,x2,x3):
#         tmp0 = [0]*len(labels2) 
#         tmp0[labels2.index(x1)] = 1
#         tmp0[labels2.index(x2)] = 1
#         tmp0[labels2.index(x3)] = 1        
#         return tmp0    
    
#     y_tmp=[]
#     for x1, x2, x3 in zip(y, y2, y3):
#         y_tmp.append( to_multi_index(x1,x2,x3) )
#     y = y_tmp

#     print(np.sum(np.array(y),axis=1))
    #print(y[:,np.newaxis])
    
#     dataset = tf.estimator.inputs.numpy_input_fn({'x': x, 'class_weights': weights}, np.array(y), shuffle=True, batch_size=500, num_epochs=epochs)
    
    dataset = tf.estimator.inputs.numpy_input_fn({'x': x, 'class_weights': weights}, y[:,np.newaxis], shuffle=True, batch_size=500, num_epochs=epochs)
    return dataset

def get_test_inputs():
    X_ = data.X_test_dense
    x = X_

    
    data.Y2_train["Seccion2"] = data.Y2_train["Seccion2"].replace(to_replace="Goles", value='Deportes')
    
    Y_ = data.Y_test
    Y2_ = data.Y2_test   
    Y3_ = data.Y3_test
    Y_ = Y_['Seccion'].reset_index(drop=True)
    Y2_ = Y2_['Seccion2'].reset_index(drop=True)
    Y3_ = Y3_['Seccion3'].reset_index(drop=True)
    
    y = Y_.values
    y2 = Y2_.values
    y3 = Y3_.values    
    
    # computing lengths of each class
    sizes = {label: Y_[Y_ == label].shape[0] for label in labels}
#     print(sizes)

    # creating weights for each sample
    scale_factor = 10e3
    weights = np.asarray([1.0/sizes[label] for label in y])
    weights = scale_factor*weights
    weights = weights[:,np.newaxis]
    print('weights shape: {}'.format(weights.shape))

#     labels2 = list( set(y) )

#     def to_multi_index(x1,x2,x3):
#         tmp0 = [0]*len(labels2) 
#         tmp0[labels2.index(x1)] = 1
#         tmp0[labels2.index(x2)] = 1
#         tmp0[labels2.index(x3)] = 1        
#         return tmp0    
    
#     y_tmp=[]
#     for x1, x2, x3 in zip(y, y2, y3):
#         y_tmp.append( to_multi_index(x1,x2,x3) )
#     y = y_tmp

#     dataset = tf.estimator.inputs.numpy_input_fn({'x': x, 'class_weights': weights}, np.array(y), shuffle=False)    
    
    dataset = tf.estimator.inputs.numpy_input_fn({'x': x, 'class_weights': weights}, y[:,np.newaxis], shuffle=False)
    return dataset


# Training



In [33]:
validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    input_fn= get_test_inputs(),
    every_n_steps=500,
    #early_stopping_metric="accuracy",
    early_stopping_metric="loss",
    early_stopping_metric_minimize=True,
    early_stopping_rounds=7000
    )

#classifier.fit(input_fn=get_train_inputs(), monitors=[validation_monitor], steps=epochs, max_steps=None)
classifier.fit(input_fn=get_train_inputs(), monitors=[validation_monitor], max_steps=None)

weights shape: (47784, 1)
Paso por aca.........................
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./tmp5/model.ckpt-11000
INFO:tensorflow:Saving checkpoints for 11001 into ./tmp5/model.ckpt.
INFO:tensorflow:Starting evaluation at 2018-08-16-13:39:42
INFO:tensorflow:Restoring parameters from ./tmp5/model.ckpt-11001
INFO:tensorflow:Finished evaluation at 2018-08-16-13:39:46
INFO:tensorflow:Saving dict for global step 11001: accuracy = 0.69313586, global_step = 11001, loss = 0.86225134
INFO:tensorflow:Validation (step 11001): loss = 0.86225134, accuracy = 0.69313586, global_step = 11001
INFO:tensorflow:loss = 1.117412, step = 11001
INFO:tensorflow:global_step/sec: 10.8447
INFO:tensorflow:loss = 0.609958, step = 11101 (4.574 sec)
INFO:tensorflow:global_step/sec: 22.2254
INFO:tensorflow:loss = 0.58190197, step = 11201 (4.499 sec)
INFO:tensorflow:global_step/sec: 22.969
INFO:tensorflow:loss = 0.57942396, step = 11301 (4.354 sec)
INFO:tensor

DNNEstimator(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x7f9f29f70b00>, 'hidden_units': [2000, 1000, 100], 'feature_columns': [_RealValuedColumn(column_name='x', dimension=1000, default_value=None, dtype=tf.float32, normalizer=None)], 'optimizer': None, 'activation_fn': <function relu at 0x7fa0236db2f0>, 'dropout': 0.8, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

# Evaluation performance


In [ ]:
score = classifier.evaluate(input_fn=get_test_inputs,steps=1)

In [ ]:
def test_inputs():
    X_ = data.X_test_dense
    x = X_
    

    
    data.Y2_train["Seccion2"] = data.Y2_train["Seccion2"].replace(to_replace="Goles", value='Deportes')
    
    Y_ = data.Y_test
    Y2_ = data.Y2_test   
    Y3_ = data.Y3_test
    Y_ = Y_['Seccion'].reset_index(drop=True)
    Y2_ = Y2_['Seccion2'].reset_index(drop=True)
    Y3_ = Y3_['Seccion3'].reset_index(drop=True)
    
    y = Y_.values
    y2 = Y2_.values
    y3 = Y3_.values    
    
    # computing lengths of each class
    sizes = {label: Y_[Y_ == label].shape[0] for label in labels}
#     print(sizes)

    # creating weights for each sample
    scale_factor = 10e3
    weights = np.asarray([1.0/sizes[label] for label in y])
    weights = scale_factor*weights
    weights = weights[:,np.newaxis]
    print('weights shape: {}'.format(weights.shape))

    labels2 = list( set(y) )

    def to_multi_index(x1,x2,x3):
        tmp0 = [0]*len(labels2) 
        tmp0[labels2.index(x1)] = 1
        tmp0[labels2.index(x2)] = 1
        tmp0[labels2.index(x3)] = 1        
        return tmp0    
    
    y_tmp=[]
    for x1, x2, x3 in zip(y, y2, y3):
        y_tmp.append( to_multi_index(x1,x2,x3) )
    y = y_tmp    
    
    x = tf.constant(x)
    class_weights =tf.constant(weights) 
    y = tf.constant(np.array(y))    
    return {'x':x,'class_weights': class_weights}, y

In [ ]:
score = classifier.evaluate(input_fn=test_inputs,steps=1)

In [ ]:
predictions = list(classifier.predict(input_fn = test_inputs))


In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_true=y_test, y_pred=predictions)
print('Accuracy in test: {}'.format(acc))

In [ ]:
#predictions[1:10]

out = list(map(lambda x:x["classes"] ,predictions))
sums = np.sum(out,axis = 1)
total = sum(sums==3)
print(total)
print(len(sums))


In [ ]:
predictions.

In [ ]:
from sklearn import cross_validation
import sklearn
sklearn.metrics.confusion_matrix(out, out)

In [ ]:
def input_fn_evaluate():
    dataset = {'x': tf.constant(data.X_test_dense)}
    
    return dataset

pred_test = classifier.predict(input_fn=input_fn=get_test_inputs)

# print(pred_test)

y_test_hat = np.asarray([x.decode('UTF-8') for x in list(pred_test)])
y_test_hat = y_test_hat.astype(str)
y_test = data.Y_test['Seccion'].values
y_test = y_test.astype(str)
print('Test shape: {}\nReal shape: {}'.format(y_test_hat.shape, y_test.shape))

from sklearn.metrics import accuracy_score

acc = accuracy_score(y_true=y_test, y_pred=y_test_hat)
print('Accuracy in test: {}'.format(acc))



In [ ]:

pred_test = list(classifier.predict(input_fn = get_test_inputs))
print(predictions[1:10])

In [ ]:
out = list(map(lambda x:x["classes"] ,predictions))


In [ ]:
out = list(map(lambda x:x["classes"] ,predictions))
out = np.array(out)

import numpy as np
import itertools
import matplotlib.pyplot as plt

def confusion_matrix(yt, yp, classes):
    instcount = yt.shape[0]
    n_classes = classes.shape[0]
    mtx = np.zeros((n_classes, 4))
    for i in range(instcount):
        for c in range(n_classes):
            mtx[c,0] += 1 if yt[i,c]==1 and yp[i,c]==1 else 0
            mtx[c,1] += 1 if yt[i,c]==1 and yp[i,c]==0 else 0
            mtx[c,2] += 1 if yt[i,c]==0 and yp[i,c]==0 else 0
            mtx[c,3] += 1 if yt[i,c]==0 and yp[i,c]==1 else 0
    mtx = [[m0/(m0+m1), m1/(m0+m1), m2/(m2+m3), m3/(m2+m3)] for m0,m1,m2,m3 in mtx]
    plt.figure(num=None, figsize=(5, 15), dpi=100, facecolor='w', edgecolor='k')
    plt.imshow(mtx, interpolation='nearest',cmap='Blues')
    plt.title("title")
    tick_marks = np.arange(n_classes)
    plt.xticks(np.arange(4), ['1 - 1','1 - 0','0 - 0','0 - 1'])
    plt.yticks(tick_marks, classes)
    for i, j in itertools.product(range(n_classes), range(4)):
        plt.text(j, i, round(mtx[i][j],2), horizontalalignment="center")

    #plt.tight_layout()
    plt.ylabel('labels')
    plt.xlabel('Predicted')
    plt.show()

confusion_matrix(out, out,[1 1 1 1 1 1 1 1 1])
    
# y1 = np.genfromtxt('y_true.csv', delimiter=",")
# y2 = np.genfromtxt('y_pred.csv', delimiter=",")
# labels = np.genfromtxt('labels.csv', delimiter=",", dtype="|S")
# confusion_matrix(y1, y2, labels)

In [ ]:
y_test = data.Y_test['Seccion'].values
y_test = y_test.astype(str)
print(y_test)

In [ ]:
x,y =get_train_inputs()

In [ ]:
#with sess.as_default()


In [ ]:
predict_proba = classifier.predict_proba(input_fn=input_fn_evaluate,as_iterable=False)
np.sum(predict_proba, axis=1)

In [ ]:

ax = skplt.plot_confusion_matrix(y_test, y_test_hat,normalize=True)
# plt.xticks(rotation=45)

for tick in ax.xaxis.get_minor_ticks():
    tick.tick1line.set_markersize(0)
    tick.tick2line.set_markersize(0)
    tick.label1.set_horizontalalignment('center')

plt2.xticks(rotation=70)